In [1]:
# Import dependecies
import pandas as pd
import numpy as np
import difflib
import re

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
movies_df = pd.read_csv('../data/movies.csv')
movies_df.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [3]:
movies_df.columns.to_list()

['index',
 'budget',
 'genres',
 'homepage',
 'id',
 'keywords',
 'original_language',
 'original_title',
 'overview',
 'popularity',
 'production_companies',
 'production_countries',
 'release_date',
 'revenue',
 'runtime',
 'spoken_languages',
 'status',
 'tagline',
 'title',
 'vote_average',
 'vote_count',
 'cast',
 'crew',
 'director']

In [4]:
# Create subset of the movies_df
movies_subset_df = pd.concat([movies_df['genres'], movies_df['title'], movies_df['cast'], movies_df['release_date'], movies_df['director'],
                             movies_df['vote_average'], movies_df['popularity']], axis=1)
movies_subset_df

,genres,title,cast,release_date,director,vote_average,popularity
0,Action Adventure Fantasy Science Fiction,Avatar,Sam Worthington Zoe Saldana Sigourney Weaver S...,2009-12-10,James Cameron,7.2,150.437577
1,Adventure Fantasy Action,Pirates of the Caribbean: At World's End,Johnny Depp Orlando Bloom Keira Knightley Stel...,2007-05-19,Gore Verbinski,6.9,139.082615
2,Action Adventure Crime,Spectre,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,2015-10-26,Sam Mendes,6.3,107.376788
3,Action Crime Drama Thriller,The Dark Knight Rises,Christian Bale Michael Caine Gary Oldman Anne ...,2012-07-16,Christopher Nolan,7.6,112.312950
4,Action Adventure Science Fiction,John Carter,Taylor Kitsch Lynn Collins Samantha Morton Wil...,2012-03-07,Andrew Stanton,6.1,43.926995
...,...,...,...,...,...,...,...
4798,Action Crime Thriller,El Mariachi,Carlos Gallardo Jaime de Hoyos Peter Marquardt...,1992-09-04,Robert Rodriguez,6.6,14.269792
4799,Comedy Romance,Newlyweds,Edward Burns Kerry Bish\u00e9 Marsha Dietlein ...,2011-12-26,Edward Burns,5.9,0.642552
4800,Comedy Drama Romance TV Movie,"Signed, Sealed, Delivered",Eric Mabius Kristin Booth Crystal Lowe Geoff G...,2013-10-13,Scott Smith,7.0,1.444476
4801,NaN,Shanghai Calling,Daniel Henney Eliza Coupe Bill Paxton Alan Ruc...,2012-05-03,Daniel Hsia,5.7,0.857008


In [5]:
movies_subset_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   genres        4775 non-null   object 
 1   title         4803 non-null   object 
 2   cast          4760 non-null   object 
 3   release_date  4802 non-null   object 
 4   director      4773 non-null   object 
 5   vote_average  4803 non-null   float64
 6   popularity    4803 non-null   float64
dtypes: float64(2), object(5)
memory usage: 262.8+ KB


In [6]:
def clean_release_date(date):
    # return the year
    return str(date).split('-')[0]

movies_subset_df['release_date'] = movies_subset_df['release_date'].apply(clean_release_date)
movies_subset_df

,genres,title,cast,release_date,director,vote_average,popularity
0,Action Adventure Fantasy Science Fiction,Avatar,Sam Worthington Zoe Saldana Sigourney Weaver S...,2009,James Cameron,7.2,150.437577
1,Adventure Fantasy Action,Pirates of the Caribbean: At World's End,Johnny Depp Orlando Bloom Keira Knightley Stel...,2007,Gore Verbinski,6.9,139.082615
2,Action Adventure Crime,Spectre,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,2015,Sam Mendes,6.3,107.376788
3,Action Crime Drama Thriller,The Dark Knight Rises,Christian Bale Michael Caine Gary Oldman Anne ...,2012,Christopher Nolan,7.6,112.312950
4,Action Adventure Science Fiction,John Carter,Taylor Kitsch Lynn Collins Samantha Morton Wil...,2012,Andrew Stanton,6.1,43.926995
...,...,...,...,...,...,...,...
4798,Action Crime Thriller,El Mariachi,Carlos Gallardo Jaime de Hoyos Peter Marquardt...,1992,Robert Rodriguez,6.6,14.269792
4799,Comedy Romance,Newlyweds,Edward Burns Kerry Bish\u00e9 Marsha Dietlein ...,2011,Edward Burns,5.9,0.642552
4800,Comedy Drama Romance TV Movie,"Signed, Sealed, Delivered",Eric Mabius Kristin Booth Crystal Lowe Geoff G...,2013,Scott Smith,7.0,1.444476
4801,NaN,Shanghai Calling,Daniel Henney Eliza Coupe Bill Paxton Alan Ruc...,2012,Daniel Hsia,5.7,0.857008


In [7]:
movies_subset_df.isna().sum()

genres          28
title            0
cast            43
release_date     0
director        30
vote_average     0
popularity       0
dtype: int64